In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

BASE_DIR = "/content/drive/MyDrive/Class Label"

print(os.listdir(BASE_DIR))
print("Train folders:", os.listdir(os.path.join(BASE_DIR, "train")))


['test', 'train', 'val']
Train folders: ['Anthrancnose', 'Healthy', 'fungal', 'Bacterial Blight', 'Root rot', 'Leaf Spot']


In [ ]:
!pip -q install tensorflow


In [ ]:
import os
import tensorflow as tf

BASE_DIR = "/content/drive/MyDrive/Class Label"  # change only if your folder name is different

train_dir = os.path.join(BASE_DIR, "train")
val_dir   = os.path.join(BASE_DIR, "val")
test_dir  = os.path.join(BASE_DIR, "test")

IMG_SIZE = (224, 224)
BATCH_SIZE = 8

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_ds.class_names


Found 1 files belonging to 6 classes.
Found 6 files belonging to 6 classes.
Found 5 files belonging to 6 classes.


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(500).prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)
test_ds  = test_ds.prefetch(AUTOTUNE)


In [ ]:
from tensorflow.keras import layers, models
import tensorflow as tf

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.12),
    layers.RandomZoom(0.12),
    layers.RandomContrast(0.12),
])

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # freeze (important for small dataset)

inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(len(class_names), activation="softmax")(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(train_ds, validation_data=val_ds, epochs=15)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step - accuracy: 0.0000e+00 - loss: 1.0850 - val_accuracy: 0.0000e+00 - val_loss: 2.0000
Epoch 2/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 2.1205 - val_accuracy: 0.0000e+00 - val_loss: 2.0060
Epoch 3/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 713ms/step - accuracy: 0.0000e+00 - loss: 4.0899 - val_accuracy: 0.0000e+00 - val_loss: 2.0137
Epoch 4/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 698ms/step - accuracy: 0.0000e+00 - loss: 1.9202 - val_accuracy: 0.0000e+00 - val_loss: 2.0229
Epoch 5/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 1.0167 - val_accuracy: 0.1667 - val_loss: 2.0324
Epoch 6/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 954ms/step - accuracy: 0.0000e+00 - loss: 2.1573 - val_accuracy: 0.1667 - val_loss: 2.0434
Epoch 7/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 928ms/step - accuracy: 0.0000e+00 - loss: 1.6452 - val_accuracy: 0.1667 - val_loss: 2.0557
Epoch 8/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print("✅ Test accuracy:", test_acc)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4000 - loss: 1.4509
✅ Test accuracy: 0.4000000059604645


In [ ]:
MODEL_PATH = "/content/drive/MyDrive/anthurium_disease_model.keras"
model.save(MODEL_PATH)
print("✅ Saved model to:", MODEL_PATH)


✅ Saved model to: /content/drive/MyDrive/anthurium_disease_model.keras


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Leaf Spot-12.jfif to Leaf Spot-12.jfif


In [ ]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

# find the uploaded filename
img_path = list(uploaded.keys())[0]
print("Using image:", img_path)

img = load_img(img_path, target_size=IMG_SIZE)
arr = img_to_array(img)
arr = np.expand_dims(arr, axis=0)

pred = model.predict(arr)[0]
idx = int(np.argmax(pred))
disease = class_names[idx]
confidence = float(np.max(pred)) * 100

print("✅ Predicted disease:", disease)
print("✅ Confidence:", round(confidence, 2), "%")

# Safety: if low confidence
if confidence < 60:
    print("⚠️ Low confidence: try a clearer close-up leaf image.")


Using image: Leaf Spot-12.jfif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
✅ Predicted disease: fungal
✅ Confidence: 65.98 %
